In [73]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entire‰ly on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime
import time

## Group Assignment
### Team Number: 04
### Team Member Names: Arav Talati, Iris Hu, Nyra Rodrigues
### Team Strategy Chosen: Market Beat

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.


In [74]:
stock_csv = "Tickers_Example.csv"

# Read the stock symbols from CSV
stocks = pd.read_csv(stock_csv, header=None)
stocks.columns = ['Column1']

start_date = "2023-10-01"
end_date = "2024-09-30"

stock_list = []

# Loop through each ticker
for tckr in stocks["Column1"]:
    try: 
        stock = yf.Ticker(tckr)

        # Check if the stock has 'currency' in the info
        if "currency" in stock.info.keys():
            stock_currency = stock.info["currency"]
            
            # Get the historical data for the ticker with monthly intervals
            stock_hist = stock.history(start=start_date, end=end_date, interval="1mo")
            time.sleep(0.3)

            # Calculate the average volume over the valid months
            if not stock_hist.empty:  # If there's data after dropping NaN volumes
                stock_volume = stock_hist['Volume'].dropna().mean()

                # Check if currency is USD or CAD and if the volume is above 100,000
                if (stock_currency == "USD" or stock_currency == "CAD") and stock_volume > 100000:
                    stock_list.append(tckr)
    except:
        pass


Since we have chosen the market beat strategy, we want to maximize our returns. We also want minimum diversification. 

Beta calculation:
Beta value measures a stock's volatility and sensitivity to market movements relative to an index. A Beta greater than 1 means the stock tends to have higher volatility, while a Beta less than 1 indicates that the stock is less volatile than the market. 

We get a list of the stocks that have the highest beta. This gives us a portfolio that is positively correlated and volatile compared to the market. This also implies that the portfolio has the least market diversification and the highest volume of change. 

In [75]:
start_date = "2024-09-01"
end_date = datetime.today()

# Defining market indices to compare against
MarketIndex1 = "^GSPTSE"  # TSX 60 #confirm ticker
MarketIndex2 = "^GSPC"  # S&P 500

# Retrieving historical close prices for the market indices
market_hist1 = yf.Ticker(MarketIndex1).history(start=start_date, end=end_date)['Close']
market_hist2 = yf.Ticker(MarketIndex2).history(start=start_date, end=end_date)['Close']

# Calculating daily percentage returns for the market indices
market_hist1["Returns"] = market_hist1.pct_change() * 100
market_hist2["Returns"] = market_hist2.pct_change() * 100

# Calculating the average daily returns of the two market indices
market_avg_returns = (market_hist1["Returns"] + market_hist2["Returns"]) / 2

# Calculating the variance of the average market returns
market_var = market_avg_returns.var()

# beta_val returns the beta of the ticker inputted
# Beta measures a stock's volatility relative to the market
def beta_val(stock_returns):
    df = pd.DataFrame({"Market Returns": market_avg_returns, "Stock Returns": stock_returns}).dropna()

    # Calculating the covariance between stock returns and market returns
    covariance = df.cov().iloc[0, 1]

    # Return the ratio of covariance to market variance (beta)
    return covariance / market_var


# betas_list takes a list of tickers and returns a dictionary with tickers as keys and their betas as values
def betas_list(ticker_list):
    betas = {} # Initialize an empty dictionary to store betas

    # Looping through each ticker in the list
    for ticker in ticker_list:
        stock_hist = yf.Ticker(ticker).history(start=start_date, end=end_date)['Close']
        time.sleep(0.3)

        # Calculate daily percentage returns for the stock
        ticker_returns = stock_hist.pct_change()*100 
        betas[ticker] = beta_val(ticker_returns)

    # Return a dictionary of betas
    return betas

# Calculate beta values for all stocks in the stock_list
all_betas = betas_list(stock_list)

In [76]:
beta_dict = betas_list(all_betas.keys())

# Convert the dictionary of beta values into a DataFrame
beta_df = pd.DataFrame.from_dict(beta_dict, orient='index')

# Rename the column in the DataFrame to 'Beta'
beta_df.columns = ['Beta']

# Set the index name of the DataFrame to 'Ticker'
beta_df.index.name = 'Ticker'

In [77]:
#Sorting the dictionary from lowest beta to highest beta
sorted_beta_df = beta_df.sort_values('Beta').copy()

# Checking if there are fewer than 12 stocks with a beta value >= 1
if (len(sorted_beta_df.loc[sorted_beta_df['Beta'] >= 1].index) < 12):
    # If there are fewer than 12 stocks, select the 15 stocks with the highest beta values
    eligible_stocks = sorted_beta_df.iloc[-15:]

# Checking if there are more than 30 stocks with a beta value >= 1
elif ((len(sorted_beta_df.loc[sorted_beta_df['Beta'] >= 1].index) > 30)): 
    # If there are more than 30 stocks, select the 30 stocks with the highest beta values
    eligible_stocks = sorted_beta_df.iloc[-30:] 
else:
    # Otherwise select all stocks with beta values >= 1
    eligible_stocks = sorted_beta_df.loc[sorted_beta_df['Beta'] >= 1]

Sharpe Ratio:
Sharpe Ratio measures the return of an investment relative to its risk. A higher Sharpe ratio indicates a better risk-adjusted return, meaning the stock or portfolio is delivering more reward for the risk taken. It helps identify stocks that offer the best potential for growth, with minimal unnecessary risk.

In [78]:
# calculate_sharpe_ratio calculates the Sharpe Ratio for a given stock ticker and risk free rate
def calculate_sharpe_ratio(ticker, risk_free_rate):

    # Getting historical closing prices for the stock
    stock_data = yf.Ticker(ticker).history(start=start_date, end=end_date)['Close']
    time.sleep(0.3)

    # Calculating daily percentage returns for the stock
    stock_returns = stock_data.pct_change()

    # Calculating the average daily return of the stock
    average_return = stock_returns.mean()

    # Calculating the standard deviation of the stock's daily returns
    std_deviation = stock_returns.std()

    # Calculating the Sharpe Ratio using the formula: (Average Return - Risk-Free Rate) / Standard Deviation
    sharpe_ratio = (average_return - risk_free_rate) / std_deviation
    return sharpe_ratio

# Initializing a dictionary to store the Sharpe Ratio
sharpe_ratios = {}

# Looping through each eligible stock ticker and calculating its Sharpe Ratio
for ticker in eligible_stocks.index:
    sharpe_ratios[ticker] = calculate_sharpe_ratio(ticker, 0)

# Adding the calculated Sharpe Ratios to the eligible_stocks DataFrame
for ticker, sharpe_ratio in sharpe_ratios.items():
    eligible_stocks.loc[ticker, "Sharpe Ratio"] = sharpe_ratio

# Sorting the eligible stocks by their Sharpe Ratio in descending order
eligible_stocks = eligible_stocks.sort_values('Sharpe Ratio', ascending=False)

# Checking if there are fewer than 12 stocks with a positive Sharpe Ratio
if (len(eligible_stocks[eligible_stocks['Sharpe Ratio'] > 0]) < 12):
    # If there are fewer than 12 stocks that have Sharpe Ratios greater than 0, select the top 12 stocks 
    eligible_stocks = eligible_stocks.iloc[:12]
else:
    # Otherwise drop stocks with a Sharpe Ratio less than 0 from the DataFrame
    lowSharpe = eligible_stocks[eligible_stocks['Sharpe Ratio'] <= 0].index
    eligible_stocks.drop(lowSharpe, inplace=True)

/var/folders/c0/qh7w74pn7x5f5q81vn0wf7480000gn/T/ipykernel_12824/939839452.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eligible_stocks.loc[ticker, "Sharpe Ratio"] = sharpe_ratio


In [79]:
# Looping through each stock ticker in the eligible_stocks DataFrame
for tckr in eligible_stocks.index:
    try:
        ticker = yf.Ticker(tckr)
        ticker_hist = ticker.history(start=start_date, end=end_date)
        time.sleep(0.3)

        # Calculating the short-term Exponential Moving Average (EMA) with a span of 2
        ticker_hist['EMA Short'] = ticker_hist['Close'].ewm(span=2, adjust=True, min_periods=5).mean()

        # Calculating the long-term Exponential Moving Average (EMA) with a span of 50
        ticker_hist['EMA Long'] = ticker_hist['Close'].ewm(span=50, adjust=True, min_periods=5).mean()

        if ticker_hist.empty:
            eligible_stocks.loc[tckr, 'Variance in EMA'] = -10000  # Assigning an extreme negative value for sorting

        else:
            # Calculating the variance between the last short-term and long-term EMA values
            eligible_stocks.loc[tckr, 'Variance in EMA'] = ticker_hist['EMA Short'].iloc[len(ticker_hist)-1] - ticker_hist['EMA Long'].iloc[len(ticker_hist)-1]
    except:
        pass

# Sorting the eligible stocks in descending order of 'Variance in EMA'
eligible_stocks = eligible_stocks.sort_values('Variance in EMA', ascending=False)

# Checking if there are less than 12 stocks with a 'Variance in EMA' greater than -2
if (len(eligible_stocks[eligible_stocks['Variance in EMA'] > -2]) < 12):

    # If less than 12 stocks meet the condition, select the top 12 stocks
    eligible_stocks = eligible_stocks.iloc[:12]

# Check if there are more than 24 stocks with a 'Variance in EMA' greater than -2
elif (len(eligible_stocks[eligible_stocks['Variance in EMA'] > -2]) > 24):

    # If more than 24 stocks meet the condition, select the top 24 stocks
    eligible_stocks = eligible_stocks.iloc[:24]
else:

    #Otherwise, drop stocks with 'Variance in EMA' less than or equal to -2
    lowReturns = eligible_stocks[eligible_stocks['Variance in EMA'] <= -2].index
    eligible_stocks.drop(lowReturns, inplace=True)

# Droping any rows with missing values from the eligible_stocks DataFrame
eligible_stocks.dropna(axis=0, inplace=True)

Price-to-Earnings (P/E) Ratio:
This is used for assessing whether a stock is overvalued or undervalued relative to its earnings. A high P/E ratio could indicate that a stock is overvalued or that investors are expecting high growth in the future. On the other hand, a low P/E ratio might suggest that a stock is undervalued or underperforming. 

By filtering for stocks with a reasonable P/E ratio, we can avoid those that are overpriced relative to their earnings potential. This ensures that our portfolio isn't overly exposed to high-risk, high-valuation stocks.

(It is also useful for comparing stocks within the same industry or sector. Different industries can have different average P/E ratios due to varying growth rates, capital requirements, and market conditions. )

In [80]:
# Dictionary mapping sector names to their respective ETF tickers
sector_etfs = {
    "Technology": "XLK",
    "Healthcare": "XLV",
    "Financial Services": "XLF",
    "Consumer Cyclical": "XLY",
    "Energy": "XLE",
    "Utilities": "XLU",
    "Consumer Defensive": "XLP",
    "Industrials": "XLB",
    "Real Estate": "XLRE",
    "Communication Services": "XLC"
}

# Function to get P/E ratio for sector ETFs
def get_sector_pe_data(sector_etfs):
    sector_pe_data = {}

    # Looping through each sector and its respective ETF ticker
    for sector, etf in sector_etfs.items():
        
        # Fetch ETF data
        sector_etf = yf.Ticker(etf)
        info = sector_etf.info
        
        # Get the P/E ratio of the sector ETF
        pe_ratio = info.get('trailingPE', None)
        
        if pe_ratio:
            sector_pe_data[sector] = pe_ratio
    
    return sector_pe_data

# Get sector P/E data for each ETF
sector_pe_data = get_sector_pe_data(sector_etfs)

In [81]:
# get_stock_pe gets the P/E ratio and sector for a list of tickers
def get_stock_pe(tickers):
    stock_data = {}

    # Looping through each ticker in the list of tickers
    for ticker in tickers:
        
        stock = yf.Ticker(ticker)

        # Get stock info
        info = stock.info

        # Get the sector of the stock and default to 'N/A' if it is not available
        sector = info.get('sector', 'N/A')
        pe_ratio = info.get('trailingPE', None)  # trailing P/E ratio
        
        # Add data to dictionary
        if (sector != 'N/A'): 

            # If P/E ratio is available, add sector and P/E to the dictionary
            if pe_ratio:
                stock_data[ticker] = {'sector': sector, 'PE': pe_ratio}
                
            else:
                # Otherwise, set P/E ratio to None
                stock_data[ticker] = {'sector': sector, 'PE': None}
        else:

            # If sector is 'N/A' and the P/E ratio is available
            if pe_ratio:
                stock_data[ticker] = {'sector': 'N/A', 'PE': pe_ratio}

            else:
                 # Otherwise, set P/E ratio to None
                stock_data[ticker] = {'sector': 'N/A', 'PE': None}

    # Returning the dictionary containing the sector and P/E ratio
    return stock_data

# Calling the function for all eligible stocks
stock_pe = get_stock_pe(eligible_stocks.index)

In [82]:
# compare_pe_to_sector compares the P/E ratio of individual stocks to their sector's average P/E ratio
def compare_pe_to_sector(stock_data, sector_pe_data):

    # Looping through each stock in the stock_data dictionary
    for tckr, data in stock_data.items():

        # Extracting the sector and P/E ratio of the stock
        sector = data['sector']
        pe = data['PE']
        
        # If the stock has a valid sector and P/E ratio
        if sector != 'N/A' and pe is not None:

            # Getting the average P/E ratio for the sector from sector_pe_data
            pe_sector_avg = sector_pe_data.get(sector, None)

            # If the sector's P/E ratio is available then calculate the difference between stock's P/E and sector's P/E
            if pe_sector_avg is not None:
                eligible_stocks.loc[tckr, "comparisonPE"] = pe - pe_sector_avg

        # If no valid sector or P/E ratio is found, set comparisonPE to 0
        else:
            eligible_stocks.loc[tckr, "comparisonPE"] = 0

# Calling the function to compare each stock's P/E ratio to its sector's P/E ratio
compare_pe_to_sector(stock_pe, sector_pe_data)

for tckr in eligible_stocks.index:

    # If the comparisonPE value is 0, replace it with the median of the column
    if eligible_stocks.loc[tckr, "comparisonPE"] == 0:
        eligible_stocks.loc[tckr, "comparisonPE"] = eligible_stocks["comparisonPE"].median()

eligible_stocks

,Beta,Sharpe Ratio,Variance in EMA,comparisonPE
Ticker,,,,
SHOP.TO,2.310890,0.234811,24.748974,56.061264
AXP,1.781327,0.177995,18.831406,-0.867416
CAT,1.941066,0.151246,8.202756,0.337385
PYPL,1.360593,0.178850,4.622066,-2.263453
C,1.892471,0.124664,4.274274,-3.124392
USB,1.561527,0.134922,3.453441,-6.871056
BAC,1.539834,0.162138,3.414824,-5.992840
AMZN,1.989800,0.113399,2.173286,8.067067
ACN,1.078936,0.064783,2.002763,-12.936858


In [83]:
# Initializing a new column "Ranking" in the eligible_stocks DataFrame with a default value of 0
eligible_stocks['Ranking'] = 0

# Getting the length of the eligible_stocks DataFrame
length = len(eligible_stocks.index)

# Sorting based on "Beta"
eligible_stocks = eligible_stocks.sort_values("Beta", ascending=False)

# Assigning a rank based on the sorted "Beta" values
for i in range(length, 0, -1):
    eligible_stocks.loc[eligible_stocks.index[length-i], 'Ranking'] += i

# Sorting based on "Sharpe Ratio"
eligible_stocks = eligible_stocks.sort_values("Sharpe Ratio", ascending=False)

# Assigning a rank based on the sorted "Sharpe Ratio" values
for i in range(length, 0, -1):
    eligible_stocks.loc[eligible_stocks.index[length-i], 'Ranking'] += i

# Sorting based on "Variance in EMA"
eligible_stocks = eligible_stocks.sort_values("Variance in EMA", ascending=False)

# Assigning a rank based on the "Variance in EMA"
for i in range(length, 0, -1):
    eligible_stocks.loc[eligible_stocks.index[length-i], 'Ranking'] += i

# Sorting based on "comparisonPE"
eligible_stocks = eligible_stocks.sort_values("comparisonPE", ascending=False)

# Assigning a rank based on the "comparisonPE"
for i in range(length, 0, -1):
    eligible_stocks.loc[eligible_stocks.index[length-i], 'Ranking'] += i

# Sorting based on the updated "Ranking" in descending order
eligible_stocks = eligible_stocks.sort_values("Ranking", ascending=False)

# Returns the final sorted DataFrame with the calculated rankings
eligible_stocks

,Beta,Sharpe Ratio,Variance in EMA,comparisonPE,Ranking
Ticker,,,,,
SHOP.TO,2.310890,0.234811,24.748974,56.061264,48
CAT,1.941066,0.151246,8.202756,0.337385,37
AXP,1.781327,0.177995,18.831406,-0.867416,37
AMZN,1.989800,0.113399,2.173286,8.067067,31
PYPL,1.360593,0.178850,4.622066,-2.263453,31
C,1.892471,0.124664,4.274274,-3.124392,28
BAC,1.539834,0.162138,3.414824,-5.992840,25
USB,1.561527,0.134922,3.453441,-6.871056,24
BB.TO,2.120600,0.063053,-0.035639,-2.693923,23


In [84]:
n = len(eligible_stocks) # number of stocks
min_weight = 100 / (2 * n)  # Minimum weight percentage
max_weight = 15.0  # Maximum weight percentage
    
# Calculate initial weights
total_points = sum(eligible_stocks['Ranking']) # Total points from rankings
initial_weights = (eligible_stocks['Ranking']/total_points) * 100 # Proportional weightage as percentages
    
# Identify names below minimum threshold
below_min = pd.DataFrame()
below_min['Initial Weights'] = initial_weights[initial_weights < min_weight]
    
# Set minimum weights for those below threshold
below_min['Final Weights'] = min_weight

# Identify names above minimum threshold
above_min = pd.DataFrame()
above_min['Initial Weights'] = initial_weights[initial_weights >= min_weight]

# Calculate remaining weight to distribute among the stocks above the minimum threshold
total_min_weight = len(below_min) * min_weight
remaining_weight = 100 - total_min_weight
total_above_min = sum(above_min['Initial Weights'])
    
# Redistribute remaining weight proportionally
above_min['Final Weights'] = (above_min['Initial Weights'] / total_above_min) * remaining_weight
final_weights = np.array(above_min['Final Weights'].values.tolist()) # Converting weights to a NumPy array
above_min['Final Weights'] = np.where(final_weights > max_weight, max_weight, final_weights).tolist() # Capping weights at max_weight

# Calculating the total allocated weight after enforcing the maximum weight constraint
total_allocated = sum(above_min['Final Weights']) + sum(below_min['Final Weights'])

# If total allocated weight is less than 100, we redistribute the excess proportionally
if total_allocated < 100:
    excess = 100 - total_allocated # Calculating the excess weight to be redistributed
    non_max_weights = above_min['Final Weights'][above_min['Final Weights'] < max_weight]
    total_non_max = sum(non_max_weights) # Calculating the total weight of stocks below the max weight
    
    # Proportionally redistributing the excess weight
    additional = (non_max_weights/total_non_max) * excess
    above_min['Final Weights'] += additional

# Combine below_min and above_min dataframes
combined_weights = pd.concat([above_min, below_min])

# Ensure sum is exactly 100
remaining = 100.0
sorted_items = pd.DataFrame()
sorted_items['Final Weights'] = combined_weights['Final Weights'].sort_values(ascending=False)

# Subtracting the weights of all stocks except the first to calculate the remaining weight
for i in range(len(sorted_items)-1):
    remaining -= sorted_items.iloc[i+1, 0]
    
# Assign the remaining weight to the first item to ensure sum is exactly 100
sorted_items.iloc[0, 0] = remaining

eligible_stocks['Weights'] = sorted_items['Final Weights']

# Making sure that all stocks have at least the minimum weight
for tckr in eligible_stocks.index:
    if (eligible_stocks.loc[tckr, "Weights"] < 100/(2*n)): # Checking if the weight is below the minimum
        difference = 100/(2*n) - (eligible_stocks.loc[tckr, "Weights"]) # Calculating the difference
        eligible_stocks.loc[tckr, "Weights"] = 100/(2*n) # Set the weight to the minimum
        eligible_stocks.iloc[0, len(eligible_stocks.axes[1])-1] -= difference # Adjusting the first stock's weight to compensate

In [85]:
investment_money = 1000000  # Total investment amount

# Creating a DataFrame to store the portfolio details, using the eligible stocks
Portfolio_Final = pd.DataFrame(eligible_stocks.index)

# Adjusting the index to start from 1
Portfolio_Final.index = Portfolio_Final.index + 1

total_value = 0

# Getting the USD to CAD conversion rate using yfinance
exchange_rate = yf.Ticker("USDCAD=x")
conversion_rate = exchange_rate.info["previousClose"]

# Looping through the eligible stocks
for i in range(1, len(Portfolio_Final.index)+1):

    # Getting the ticker
    tckr = eligible_stocks.index[i-1]

    ticker = yf.Ticker(Portfolio_Final.loc[i, "Ticker"])

    # Storing the previous close price of the stock
    Close = ticker.info["previousClose"]
    Portfolio_Final.loc[i, "Price"] = Close

    # Storing the currency of the stock
    Currency = ticker.info["currency"]
    Portfolio_Final.loc[i, "Currency"] = Currency

    # Converting the price to CAD if the currency is in SD
    if (Portfolio_Final.loc[i, "Currency"] == "USD"):
        Close = Close * conversion_rate

    # Calculating the potential transaction fees
    pot_fees = ((eligible_stocks.loc[tckr, "Weights"]/100)*investment_money)/Close*0.001

    # Checking the smaller fee
    if pot_fees > 3.95:
        money = ((eligible_stocks.loc[tckr, "Weights"]/100)*investment_money) - 3.95
    else:
        money = ((eligible_stocks.loc[tckr, "Weights"]/100)*investment_money) - pot_fees

    # Storing the number and value of the shares
    Shares = money/Close
    Portfolio_Final.loc[i, "Shares"] = Shares
    Portfolio_Final.loc[i, "Value"] = Shares*Close

    # Adding the fees to the total portfolio value
    if pot_fees > 3.95:
        total_value +=  Portfolio_Final.loc[i, "Value"] + 3.95
    else:
        total_value +=  Portfolio_Final.loc[i, "Value"] + (Portfolio_Final.loc[i, "Shares"]*0.001)

# Calculating the weight of each stock as a percentage
Portfolio_Final["Weight"] = (Portfolio_Final["Value"]/sum(Portfolio_Final["Value"])) * 100

# Verifying that all the weights sum to 100%
print ("The total value of the Portfolio is: $", round(total_value))
total_weight = sum(Portfolio_Final["Weight"])
print("The total weight of each of the stocks in the portfolio add up to: ", total_weight, "%", )

Portfolio_Final

The total value of the Portfolio is: $ 1000000
The total weight of each of the stocks in the portfolio add up to:  100.0 %


,Ticker,Price,Currency,Shares,Value,Weight
1,SHOP.TO,148.81,CAD,993.792958,147886.330144,14.788786
2,CAT,389.59,USD,209.434936,113996.269438,11.399745
3,AXP,293.00,USD,278.476817,113996.200396,11.399738
4,AMZN,198.38,USD,344.602358,95510.218777,9.551120
5,PYPL,84.82,USD,805.964224,95509.757409,9.551074
6,C,68.95,USD,895.519930,86266.710105,8.626760
7,BAC,46.46,USD,1186.615770,77023.461253,7.702426
8,USB,51.39,USD,1029.870406,73942.632087,7.394340
9,BB.TO,3.26,CAD,21735.805539,70858.726056,7.085946
10,ACN,361.05,USD,82.601292,41666.584065,4.166701


Note that the weight shown in the table are not greater than 100/2n% (where n is the number of stocks in the portfolio), as we are comparing the value against the full $1000000 of investment money, which doesn't take into account the fees that impact the investment weight value.

In [86]:
# Selecting only the 'Ticker' and 'Shares' columns
Stocks_Final = Portfolio_Final[['Ticker','Shares']]
# Exporting the selected stock data to a CSV file
Stocks_Final.to_csv('Stocks_Group_04.csv')

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Arav Talati <br>
Iris Hu <br>
Nyra Rodrigues